In [8]:
import pandas as pd
import numpy as np
import sklearn as sk
import math

In [9]:
dist = pd.read_csv('dist_data.csv', sep=';')

In [217]:
dist_clean = dist[dist['dist'] != "[[], []]"]

In [218]:
x = dist_clean.iloc[:, :-1].values
y = dist_clean.iloc[:, 1].values

In [219]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

In [220]:
def clean_sample(sample):
    sample_list = sample[0].replace(', [', '; [').split(';')
    sample_x = sample_list[0]
    sample_Px = sample_list[1]

    sample_x_list = []
    sample_Px_list = []
    
    for x in sample_x.split(','):
        sample_x_list.append(x.replace(' ', '').replace('[', '').replace(']', ''))
    for Px in sample_Px.split(','):
        sample_Px_list.append(Px.replace(' ', '').replace('[', '').replace(']', ''))

    dict_sample = {}
    for i in range(len(sample_x_list)):
        dict_sample[sample_x_list[i]] = sample_Px_list[i]
    return dict_sample

In [221]:
def all_keys_dataset1_exist_in_dataset2(dataset1, dataset2):
    count = 0
    total = len(dataset1.keys())
    for key in dataset1.keys():
        if dataset2.get(key) is None:
            pass
        else:
            count += 1

    return count/total

In [223]:
# final1, final2 = {}, {}
# for i, test_value in enumerate(x_test):
#     for j, train_value in enumerate(x_train):
#         dataset1 = clean_sample(test_value)
#         dataset2 = clean_sample(train_value) 
#         value = all_keys_dataset1_exist_in_dataset2(dataset1, dataset2)
#         if value != 0.0:
#             final1.update(dataset1)
#             final2.update(dataset2)
#             print(value, y_train[j], y_test[i])
#         if len(final2) > 0:
#             break
#     break

0.29838709677419356 27321 27321


In [225]:
# from scipy import stats

# data1 = [float(value) for value in final1.values()]
# data2 = [float(value) for value in final2.values()]

# # محاسبه معیار شباهت کولموگروف-سمیرنوف
# statistic, p_value = stats.ks_2samp(data1, data2)

# # چاپ شباهت و p-value
# print("معیار شباهت کولموگروف-سمیرنوف:", statistic)
# print("p-value:", p_value)

معیار شباهت کولموگروف-سمیرنوف: 0.34465195246179964
p-value: 0.0013663862001737925


In [158]:
def CaculateEx(finall):
    E = 0
    for x, Px in finall.items():
        E += (int(x) * float(Px))
    return E

In [159]:
def CaculateEx2(finall):
    E = 0
    for x, Px in final1.items():
        E += ((int(x)**2) * float(Px))
    return E

In [160]:
print(CaculateEx(final2))
print(CaculateEx(final1))
print(CaculateEx2(final2) - (CaculateEx(final2)**2))
print(CaculateEx2(final1) - (CaculateEx(final1)**2))
print(math.sqrt(CaculateEx2(final2) - (CaculateEx(final2)**2)))
print(math.sqrt(CaculateEx2(final1) - (CaculateEx(final1)**2)))


239.69171691506688
15630.03419363291
412573781.6868164
168333264.9118401
20311.912309943058
12974.331000550283


In [161]:
# import numpy as np
# from scipy.stats import wasserstein_distance

# # دو توزیع را به شکل لیست مقدارها تعریف کنید
# dist1 = [1.1289757202627561e-05]
# dist2 = [6.161483709714915e-06]
# # تبدیل لیست های مقدارها به numpy arrays
# dist1 = np.array(dist1)
# dist2 = np.array(dist2)

# # محاسبه معیار همبستگی کرونباکر بین دو توزیع
# correlation = np.corrcoef(dist1, dist2)[0, 1]

# # محاسبه فاصله ی واسرشتاین بین دو توزیع
# wasserstein_distance = wasserstein_distance(dist1, dist2)

# print(correlation)
# print(wasserstein_distance)

nan
5.128273492912646e-06


In [ ]:
def calculate_distance(distance_func, min_precent_similarity):
    # count = 0
    result = []
    for i, sample_x_test in enumerate(x_test): 
        result_zikma = []
    
        for j, sample_x_train in enumerate(x_train):
            clean_sample(sample_x_test)
            clean_sample(sample_x_train)
            if all_keys_dataset1_exist_in_dataset2(clean_sample(sample_x_test), clean_sample(sample_x_train), min_precent_similarity):
                Px_list = []
                for xd1, Pxd1 in clean_sample(sample_x_test).items():
                    if xd1 == "" or Pxd1 == "":
                        continue
                    Pxd2 = clean_sample(sample_x_train).get(xd1)
                    if Pxd2 is None:
                        return False
                    #print(xd1, Pxd1, Pxd2)
                    Px_list.append([Pxd1, Pxd2])

                
                sum = distance_func(Px_list)
                result_zikma.append([y_train[j], sum])
                #print(Px_list, sum)
                
            else:
                continue
        
        result.append([sample_x_test ,result_zikma])
        
        # count += 1
        # if count > 10:
        #     break
    
    return result

In [ ]:
def js(Px_list):
    for i in Px_list:
        Mx = (float(i[0]) + float(i[1]))/2
        i.append(Mx)

    PM_list = []
    QM_list = []
    for i in Px_list:
        PM_list.append([i[0], i[2]])
        QM_list.append([i[1], i[2]])

    sum1 = kl(PM_list)
    sum2 = kl(QM_list)

    return (sum1 + sum2)/2

In [ ]:
def kl(Px_list):
    sum = 0 
    for i in Px_list:
        Pxd1 = i[0]
        Pxd2 = i[1]
        sum += float(Pxd1) * math.log10(float(Pxd1) / float(Pxd2))
    return sum

In [ ]:
# def Predict_test1(all_distance_label, threshold):
    # #using max
    # y_pred = []
    # for i in range(len(x_test)):
    #     t = pd.DataFrame(final_result[i][1], columns=['Label', 'Kl Result'])
    #     bests = t.value_counts()
    #     if bests.size == 0:
    #         y_pred.append(0)
    #     else:
    #         y_pred.append(bests.head(1).index[0][0])
    # return y_pred

# def Predict_test2(all_distance_label, threshold):
    # #using sort
    # y_pred = []
    # for i in range(len(x_test)):
    #     t = pd.DataFrame(final_result[i][1], columns=['Label', 'Kl Result'])
    #     grouped_df = t.groupby(['Label', 'Kl Result']).size().reset_index(name='Count')
    #     sorted_df = grouped_df.sort_values(by=['Count', 'Kl Result'], ascending=[False, True])
    #     if t.values.size == 0:
    #         y_pred.append(0)
    #     else:
    #         y_pred.append(sorted_df.head(1)['Label'].values[0])
    # return y_pred    

In [ ]:
def Predict(all_distance_label, threshold):
    y_pred = []
    for i in range(len(x_test)):
        t = pd.DataFrame(all_distance_label[i][1], columns=['Label', 'Kl Result'])
        tmp = t[t['Kl Result'] < threshold] #0.99 - 1.01
        grouped_df = tmp.groupby(['Label']).size().reset_index(name='Count')
        sorted_df = grouped_df.sort_values(by=['Count'], ascending=False)
        
        if tmp.values.size == 0:
            y_pred.append(0)
        else:
            y_pred.append(sorted_df.head(1)['Label'].values[0])
    return y_pred

In [ ]:
def diffrence_pred_test(y_pred, y_test):
    test = []
    count = 0
    for i, value1 in enumerate(y_pred):
        if value1 != y_test[i]:
            test.append([i, value1, y_test[i]])
            count+=1
    
    return pd.DataFrame(test, columns=['index', 'y_pred', 'y_test'])

In [ ]:
# def other_percent_similarity():
    # result = []
    # for i in range(1, 100): # one or zero
    #     result.append(calculate_distance(kl, i))
    # return result

In [ ]:
#final_result_kl = calculate_distance(kl, 100)

In [ ]:
y_pred_kl = Predict(final_result_kl, 1.01) #good threshold 0.99 - 1.01

In [ ]:
diffrence_pred_test(y_pred_kl, y_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
res0 = confusion_matrix(y_test, y_pred_kl)
print("Confusion Matrix:")
print(res0)
res01 = classification_report(y_test, y_pred_kl)
print("Classification Report:",)
print (res01)
res02 = accuracy_score(y_test,y_pred_kl)
print("Accuracy:",res02)

In [ ]:
#final_result_js = calculate_distance(js, 100)

In [ ]:
y_pred_js = Predict(final_result_js, 0.093) #good threshold 0.99 - 1.01

In [ ]:
diffrence_pred_test(y_pred_js, y_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
res1 = confusion_matrix(y_test, y_pred_js)
print("Confusion Matrix:")
print(res1)
res11 = classification_report(y_test, y_pred_js)
print("Classification Report:",)
print (res11)
res12 = accuracy_score(y_test,y_pred_js)
print("Accuracy:",res12)